### Reading through Python Cookbook, notedown some interesting receipe

In [15]:
# 1.8
# intersting to know dictionary can flip over key-value with zip
# afterward can also apply min/max/sort function to the result, pretty handy
prices = {
'ACME': 45.23,
'AAPL': 612.78,
'IBM': 205.55,
'HPQ': 37.20,
'FB': 10.75
} 

for val in zip(prices.values(), prices.keys()):
    print (val)

prices_sorted = sorted(zip(prices.values(), prices.keys()))
print(prices_sorted)



(45.23, 'ACME')
(612.78, 'AAPL')
(205.55, 'IBM')
(37.2, 'HPQ')
(10.75, 'FB')
[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


In [32]:
# 1.9 Dictionary Keys and Values is actually an object and can use some basic set-like ops
a = {
'x' : 1,
'y' : 2,
'z' : 3
}
b = {
'w' : 10,
'x' : 11,
'y' : 2
}

# Find keys in common
print(a.keys() & b.keys())
 # { 'x', 'y' }
# Find keys in a that are not in b
print(a.keys() - b.keys())
 # { 'z' }
# Find (key,value) pairs in common
print(a.items() & b.items()) # { ('y', 2) }


{'x', 'y'}
{'z'}
{('y', 2)}


In [35]:
# 1.20 ChainMap merge two dict without actually merging it, any change will
# be place on the first dict. It is more memory efficient than creating / mutating a existing dict
from collections import ChainMap
a = {'x': 1, 'z': 3 }
b = {'y': 2, 'z': 4 }
c = ChainMap(a,b)
print(c['x'])
print(c['y'])
print(c['z']) # only first dict value return


1
2
3


In [5]:
# 2.2 It is possible to match multiple string with startwith and endwith, 
# but the input must be a tuple. This is much prefere than the using regex 
# in term of performance

f = 'test.csv'
target_suffix = ('.txt', 'csv', 'xlsx')
f.endswith(target_suffix)

True

In [ ]:
# 2.3 string operator > fnmatch > regex
# fnmatch can be used in non filename match also

In [6]:
# 2.8 regex matching c style / sql style comment
# /\*((?:.|\n)*?)\*/
# (?:) - is for matching group but without capturing them 

In [12]:
# 2.14 It is bad to do this to concat string, thought I often do it, probably a bad habit carry over from VBA
# This will create a lot of temporary useless variable
parts = ['Hello', 'World']
s = ''
for p in parts:
    s += p
# instead imply using .join is better
print(''.join(parts))

# This chapter also provided an example of printing string base on text size using generator,
# quite interesting
def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago?'

def combine(source, maxsize):
    parts = []
    size = 0
    for part in source:
        parts.append(part)
        size += len(part)
        if size > maxsize:
            yield ''.join(parts)
            parts = []
            size = 0
        yield ''.join(parts)


HelloWorld
Is
IsChicago
IsChicagoNot
IsChicagoNotChicago?


In [1]:
# 5.2
# hmm, there is a way to print statment directly to a file like so,
# it doesn't have a lot of practical purpose, but always something new to know
with open('test.txt', 'w') as f:
    print('hi', file=f)

In [1]:
# 5.5
# this is pretty good, x-mode is like w-mode, but when file already exist it will raise an error instead of replace
with open('test.txt', 'x') as f:
    f.write('test')

FileExistsError: [Errno 17] File exists: 'test.txt'

In [8]:
# 7.5
# this recipe talks about the usual function default value for mutable object issue
# but what interesting is the solution of using an generic object as default value
_no_value = object()
def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')

spam(1) # No b value supplied
spam(1,'a') 

# this way "None" can also be a accepted optional input


No b value supplied


In [ ]:
# 7.8 functool.partial can prefix a function arguments with certain values and return a new functino with less function para
# It is essential for some function which only takes a functino with single para, such as sort() key.abs


In [ ]:
# 7.9 this provide a pretty good use case for "closure". it is often a better alternative for single method class, as 
# the purpose of a class is usually to store a state, which can be done in closure as well
# A closure is just a function with additional external environment varaible when defined

In [9]:
# 7.11 This is a pretty cool demonstration of usage of coroutine/generator with .send()
# the .send() was used to control the flow in the caller, instead of the other way around

def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

from queue import Queue
from functools import wraps
class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None) # start generator
        while True:
            result = result_queue.get()
            try:
                a = f.send(result) # send result back to outer function
                apply_async(a.func, a.args, callback=result_queue.put) # cal result and put in result_queue
            except StopIteration:
                break
    return wrapper

def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


In [ ]:
# 7.12 Adding setter and getter for clousure function. This is a lot more like a class instance, though it can be faster because no "self" variable were used